<a href="https://colab.research.google.com/github/henryouly/mssb-1099b-txf/blob/main/MSSB_PDF_TO_TEXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MSSB Pdf To Text

## 1. Colab Setup

Intall OCR library

In [ ]:
!sudo apt install poppler-utils

Restart the runtime

Importing Libraries

In [ ]:
import datetime
import re
import subprocess
import sys
from google.colab import files


## 2. Upload the PDF

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(filename)

## 3. Run PdfToText

In [ ]:
import subprocess

text = subprocess.check_output(['pdftotext', '-raw', filename, '-']).decode()

In [ ]:
print(text)

## 4. Convert to TXF

In [ ]:
txf_filename = f"{filename[:-4]}.txf"

# Codes and structure are defined at
# https://www.taxdataexchange.org/txf/txf-spec.html
categories = {
        'Short Term – Noncovered Securities': '711',
        'Long Term – Noncovered Securities': '713',
}

# Match a section of sales for one sales category.
# The last line can say 'Total Short Term – Noncovered Securities' or
# 'Total Short Term Noncovered Securities' (without the hypen) so match
# only on "^Total".
categories_pattern = '|'.join(categories)
section_expr = re.compile(
        r'('+categories_pattern+r')'
        r'(.*?)'
        r'^Total', re.DOTALL|re.MULTILINE)

# Fields: RefNumber Description CUSIP Quantity DateAcquired DateSold
#         GrossProceeds CostBasis
#
# Example:
#   1234 ALPHABET INC CL C
#   12345A678
#   1.000000 01/01/20 02/01/20 $2,000.00 $1,9999.00
#
# Example:
#   1234 ALPHABET INC CL C
#   12345A678
#   1.000000 VARIOUS 02/01/20 $2,000.00 $1,9999.00
row_expr = re.compile(
        r'^(?P<descr>(\w| )+)\s+'
        r'(?P<cusip>\w+)\s+'
        r'(?P<quantity>\d*\.\d+)\s+'
        r'(?P<acquired>(\d+/\d+/\d+|\w+))\s+'
        r'(?P<sold>\d+/\d+/\d+)\s+'
        r'(?P<proceeds>\$[0-9,.]+)\s+'
        r'(?P<cost>\$[0-9,.]+)\s', re.DOTALL|re.MULTILINE)

def parseAndPrintRows(text, entry_code, f):
    for match in row_expr.finditer(text):
        print('TD', file=f)
        print('N' + entry_code, file=f)
        print('C1', file=f)
        print('L1', file=f)
        print('P' + match.group('descr'), file=f)
        print('D' + match.group('acquired'), file=f)
        print('D' + match.group('sold'), file=f)
        # These have a leading dollar sign.
        print(match.group('cost'), file=f)
        print(match.group('proceeds'), file=f)
        print("$", file=f) # Wash sale. Leaving blank. They aren't handled here.
        print('^', file=f)

with open(txf_filename, "w") as f:
  print('V042', file=f)
  print('A mssb_1099b_to_txf', file=f)
  print('D ' + datetime.datetime.now().strftime('%m/%d/%Y'), file=f)
  print('^', file=f)
  for section_match in section_expr.finditer(text):
    entry_code = categories[section_match.group(1)]
    contents = section_match.group(2)
    parseAndPrintRows(contents, entry_code, f)
print("Finished")
files.download(txf_filename)